In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import datetime 
import statsmodels.api as sm
from sklearn import metrics

# Preprocesado y modelado
# ==============================================================================
from scipy.stats import pearsonr
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
# from factor_analyzer import FactorAnalyzer # Ánalisis factorial
from scipy.stats import bartlett
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import IncrementalPCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
import multiprocessing
# from regressors import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn import tree
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min


# Configuración matplotlib
# ==============================================================================
plt.rcParams['image.cmap'] = "bwr"
#plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')
import matplotlib.font_manager
from matplotlib import style
style.use('ggplot') or plt.style.use('ggplot')

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')


from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Models to train
forest_model = dict(model=RandomForestRegressor(), 
                        hyperparams_space=dict(n_estimators = [10, 100, 300],
                                               max_features= [2, 4, 6], 
                                               max_depth=[int(x) for x in np.linspace(2, 50, num = 10)]))
    
ridge_model = dict(model=Ridge(), hyperparams_space=dict(alpha = np.linspace(0, 1, 20)))

svr_model = dict(model=svm.SVR(), hyperparams_space = {
        'kernel': ['rbf'],
        'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5, 0.6, 0.9],
        'C': [1, 10, 100, 1000, 10000]})
    
xgb_model = dict(model=xgb.XGBRegressor(), hyperparams_space= {
    "n_estimators":[int(x) for x in np.linspace(start = 100, stop = 500, num = 10)],
        "max_depth": [int(x) for x in np.linspace(10, 110, num = 10)],
        "eta":[x for x in np.linspace(0, 1, num = 10)],
        "max_leaves": [int(x) for x in np.linspace(0, 110, num = 10)],})
    
SDG_model= dict(model=SGDRegressor(),hyperparams_space= {"penalty":["l2", "l1", "elasticnet"] ,
                                                         "alpha": [1e-7,1e-6,1e-5,1e-4, 1e-3] ,
                                                        "learning_rate":["constant","optimal","invscaling","adaptive"]})  

knn_model =  dict(model= KNeighborsRegressor(), hyperparams_space={"n_neighbors":[int(x) for x in np.linspace(1, len(dataframes[0])/3, num = 10)]})
    
    
    
Hist_model= dict(model=HistGradientBoostingRegressor(), hyperparams_space={"loss":["least_squares", "least_absolute_deviation", "poisson"]}) 


MLP_model= dict(model=MLPRegressor(),   hyperparams_space={"activation":["identity", "logistic", "tanh", "relu"],
"alpha": [1e-7,1e-6,1e-5,1e-4, 1e-3],
"learning_rate":["constant", "invscaling", "adaptive"] })



In [ ]:
def nested_cross_validation(X, y, models: list, outer_n_splits=10, inner_n_splits=3, random_state=1):
    cv_outer = KFold(n_splits=outer_n_splits, shuffle=True, random_state=random_state)
    results = {}
    results_metrics = {}
    metrics = ["mse", "rmse", "r2", "mae"]
    for model in models:
        model["name"] = model["model"].__class__.__name__
        results[model["name"]] = dict(metrics={"mse": [], "rmse": [], "r2": [], "mae": []}, best_model_score = -1)
        results_metrics[model["name"]] = {metric:[] for metric in metrics}
    for train_ix, test_ix in cv_outer.split(X):
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = y[train_ix], y[test_ix]
        cv_inner = KFold(n_splits=inner_n_splits, shuffle=True, random_state=random_state)
        for model in models:
            try:
                model["model"].set_params(**{"random_state": random_state})
            except:
                pass
            # define search
            search = RandomizedSearchCV(model["model"], 
                                               model["hyperparams_space"], 
                                               #scoring='neg_mean_squared_error', 
                                               scoring='r2',
                                               cv=cv_inner, refit=True)
            # execute search
            result = search.fit(X_train, y_train)
            # get the best performing model fit on the whole training set
            current_best_model = result.best_estimator_
            # evaluate model on the hold out dataset
            yhat = current_best_model.predict(X_test)
 
            # evaluate the model
            mse = mean_squared_error(y_test, yhat)
            mae = mean_absolute_error(y_test, yhat)
            rmse = np.sqrt(mse)
            r2 = r2_score(y_test, yhat)

            # store the result
            results_metrics[model["name"]]["mae"].append(mae)
            results_metrics[model["name"]]["mse"].append(mse)
            results_metrics[model["name"]]["rmse"].append(rmse)
            results_metrics[model["name"]]["r2"].append(r2)
                
            if r2 > results[model["name"]]["best_model_score"]:
                results[model["name"]]["best_model_score"] = r2
                results[model["name"]]["best_model_cfg"] = result.best_params_
                results[model["name"]]["best_model"] = current_best_model
            # report progress
            #print(f'>{model["name"]} mse={mse:.4f}, rmse={rmse:.4f}, cfg={result.best_params_}')
        #print()
       
        # summarize the estimated performance of the model and get the best one
        best_model_score = -1
        for model in models:
            current_metrics = results_metrics[model["name"]]["r2"]
            mean_current_metrics = np.mean(current_metrics)
            if mean_current_metrics > best_model_score:
                best_model_score = mean_current_metrics
                best_model_sd = np.std(current_metrics)
                best_model = results[model["name"]]["best_model"]
                best_model_cfg = results[model["name"]]["best_model_cfg"]
    table_result = pd.DataFrame(results)
    best_model_name = best_model.__class__.__name__
    print(f"El mejor modelo es: {best_model_name}")
    print(f"best_model_score R2 {best_model_score}")
    print(f"best_model_score STD {best_model_sd}")
    print(f"best_model selected r2  {results_metrics[best_model_name]['r2']}")
    print(f"best_model_cfg {best_model_cfg}")   
    for model in models:
        for metric in metrics:
            results_metrics[model["name"]][metric] = np.mean(results_metrics[model["name"]][metric])
    return best_model, pd.DataFrame(results_metrics)

Solo hay que seleccionar el X, Y, para probar

In [ ]:
model, result_metrics = nested_cross_validation(X, y, [forest_model,svr_model, xgb_model,SDG_model,knn_model,Hist_model,MLP_model])